<a href="https://colab.research.google.com/github/Faiz-Hibatullah/machine-learning-1103210172/blob/main/3.%20Week%203/Regression_Infrared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Setup Data

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset from GitHub raw URL
data_url = (
    'https://raw.githubusercontent.com/farrelrassya/teachingMLDL/'
    'main/02.%20Deep%20Learning/Dataset/Infrared.csv'
)
df = pd.read_csv(data_url)

# Display first rows
df.head()

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
0,Male,41-50,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525,36.59
1,Female,31-40,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075,37.19
2,Female,21-30,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850,37.34
3,Female,21-30,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825,37.09
4,Male,18-20,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175,37.04


#2. Mengeksplor Data

In [5]:
# Check shape and missing values
print("Shape:", df.shape)
print("Missing values per column:\n", df.isna().sum())

# Basic statistics
df.describe()

Shape: (1020, 34)
Missing values per column:
 Gender         0
Age            0
Ethnicity      0
T_atm          0
Humidity       0
Distance       2
T_offset1      0
Max1R13_1      0
Max1L13_1      0
aveAllR13_1    0
aveAllL13_1    0
T_RC1          0
T_RC_Dry1      0
T_RC_Wet1      0
T_RC_Max1      0
T_LC1          0
T_LC_Dry1      0
T_LC_Wet1      0
T_LC_Max1      0
RCC1           0
LCC1           0
canthiMax1     0
canthi4Max1    0
T_FHCC1        0
T_FHRC1        0
T_FHLC1        0
T_FHBC1        0
T_FHTC1        0
T_FH_Max1      0
T_FHC_Max1     0
T_Max1         0
T_OR1          0
T_OR_Max1      0
aveOralM       0
dtype: int64


,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,aveAllL13_1,T_RC1,T_RC_Dry1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
count,1020.000000,1020.000000,1018.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,...,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000
mean,24.115392,28.723039,0.729784,0.968648,35.596533,35.611474,34.888475,35.011345,35.659921,35.587143,...,34.567782,34.565340,34.487701,34.577293,35.421555,35.094144,36.084785,35.805519,35.838277,37.028382
std,1.336338,13.071627,2.456486,0.362587,0.574888,0.549760,0.718613,0.633836,0.553897,0.569278,...,0.669410,0.678663,0.668065,0.728678,0.523477,0.576045,0.491821,0.559258,0.559316,0.509502
min,20.200000,9.900000,0.540000,-0.590000,33.897500,34.122500,31.770000,32.902500,33.985000,33.825000,...,31.452500,31.657500,31.280000,31.150000,33.407500,32.440000,34.892500,33.802500,33.835000,35.540000
25%,23.400000,17.600000,0.600000,0.772500,35.247500,35.271875,34.456250,34.651250,35.332500,35.249375,...,34.180000,34.177500,34.096875,34.225000,35.116875,34.756458,35.775000,35.474375,35.500000,36.777500
50%,24.000000,26.300000,0.620000,0.940000,35.548750,35.575000,34.915000,34.997500,35.602500,35.533750,...,34.597500,34.602500,34.512917,34.622500,35.391250,35.100000,36.027500,35.790000,35.825000,36.940000
75%,24.700000,36.200000,0.700000,1.140000,35.872500,35.883125,35.300000,35.363125,35.910625,35.855625,...,34.968125,34.965000,34.878125,35.012500,35.675000,35.415000,36.280000,36.087500,36.120625,37.140000
max,29.100000,61.200000,79.000000,2.875000,38.405000,38.042500,37.575000,37.680000,38.385000,38.380000,...,37.075000,37.165000,37.212500,37.367500,38.002500,37.632500,38.807500,38.417500,38.455000,40.340000


#3. Preprocessing

In [14]:
# Separate features and target
X = df.drop(columns=['aveOralM'])
y = df['aveOralM']

# Encode categorical variables (if any) using one-hot encoding
df_encoded = pd.get_dummies(X, drop_first=True)

# Update X to the encoded DataFrame
X = df_encoded

# Impute missing values using mean strategy
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
# Fit imputer and transform, keeping column names
df_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X = df_imputed

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features for K-NN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#4. K-Nearest Neighbors Regression

K-Nearest Neighbors (K-NN) adalah algoritma supervised learning yang sederhana namun efektif untuk regresi dan klasifikasi. Pada regresi, K-NN memprediksi nilai target untuk data baru dengan cara:

1. Menghitung jarak antara data baru dan semua data latih di ruang fitur (biasanya menggunakan jarak Euclidean).

2. Memilih $k$ data latih terdekat (tetangga).

3. Mengambil rata-rata nilai target dari $k$ tetangga tersebut sebagai prediksi.



##Formulasi Matematis

Pada regresi K-Nearest Neighbors (K-NN), prediksi untuk titik query $\mathbf{x}$ adalah rata-rata nilai target $y_i$ dari $k$ tetangga terdekat dalam data latih. Langkah-langkah:

Hitung jarak Euclidean ke setiap titik latih $\mathbf{x}_i$:

$$
d(\mathbf{x}, \mathbf{x}_i) = \sqrt{\sum_{j=1}^p (x_j - x_{i,j})^2}
$$



Pilih $k$ titik dengan jarak terkecil.

Prediksi:

$$
\hat{y}(\mathbf{x}) = \frac{1}{k} \sum_{i \in \mathcal{N}_k(\mathbf{x})} y_i
$$


Dimana:

$p$ = jumlah fitur

$x_j$ = fitur ke-$j$ dari $\mathbf{x}$

$x_{i,j}$ = fitur ke-$j$ dari titik latih ke-$i$

In [15]:
# Initialize and train K-NN regressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_knn = knn.predict(X_test_scaled)

#5. Decision Tree Regression

Decision Tree adalah algoritma supervised learning yang dapat digunakan untuk tugas regresi maupun klasifikasi. Struktur model menyerupai pohon keputusan (if-else), dimana setiap node mempartisi data berdasarkan nilai fitur:

1. Pada setiap node, dipilih fitur dan ambang (threshold) yang meminimalkan galat (untuk regresi) atau impuritas (untuk klasifikasi).

2. Data dibagi menjadi dua cabang (kiri dan kanan) sesuai kondisi fitur tersebut.

3. Proses ini diulang secara rekursif hingga kriteria berhenti terpenuhi (misal: kedalaman maksimum, jumlah sampel minimum).

4. Pada daun (leaf), prediksi regresi adalah rata-rata target data di leaf tersebut.


##Formulasi Matematis

Regresi pohon keputusan mempartisi ruang fitur dengan memilih fitur $j$ dan ambang $t$ yang meminimalkan jumlah kuadrat galat dalam setiap region. Untuk suatu pembelahan:

- Region kiri: $x_i^{(j)} \le t$, rata-rata target $\mu_{kiri} = \frac{1}{n_{kiri}} \sum_{i\in kiri} y_i$

-  Region kanan: $x_i^{(j)} > t$, rata-rata target $\mu_{kanan} = \frac{1}{n_{kanan}} \sum_{i\in kanan} y_i$

Kita memilih $j$ dan $t$ yang meminimalkan:

$$
\sum_{i \in \text{kiri}} (y_i - \mu_{\text{kiri}})^2
\;+\;
\sum_{i \in \text{kanan}} (y_i - \mu_{\text{kanan}})^2
$$


Pohon dibangun secara rekursif hingga kriteria berhenti terpenuhi (misal: kedalaman maksimum, jumlah sampel minimum).

In [16]:
# Initialize and train Decision Tree regressor
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Predict on test set
y_pred_dt = dt.predict(X_test)

#6. Evaluasi

Kita menggunakan tiga metrik untuk mengevaluasi model regresi:

- Mean Squared Error (MSE):

$$
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2
$$


- Root Mean Squared Error (RMSE):

$$
\mathrm{RMSE} = \sqrt{\mathrm{MSE}}
$$


- Koefisien Determinasi ($R^2$):

$$
R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y})^2}
\quad\text{dengan}\quad
\bar{y} = \frac{1}{n} \sum_{i=1}^n y_i
$$


dimana $\bar{y} = \frac{1}{n} \sum_{i=1}^n y_i$ adalah rata-rata nilai target.

Interpretasi:

MSE/RMSE yang lebih kecil menunjukkan model lebih baik.

$R^2$ mendekati 1 berarti variansi data terjelaskan dengan baik oleh model.

In [17]:
# Define a function to compute metrics
def evaluate_model(y_true, y_pred, name="Model"):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} Performance:")
    print(f"  MSE:  {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2:   {r2:.4f}\n")

# Evaluate K-NN
evaluate_model(y_test, y_pred_knn, "K-NN Regressor")

# Evaluate Decision Tree
evaluate_model(y_test, y_pred_dt, "Decision Tree Regressor")

K-NN Regressor Performance:
  MSE:  0.0727
  RMSE: 0.2697
  R2:   0.6547

Decision Tree Regressor Performance:
  MSE:  0.1264
  RMSE: 0.3556
  R2:   0.3996

